## Importing

In [1]:
import os
import numpy as np
import cv2
import multiprocessing as mp
import time
import re
import functools
min_confidence = 0.2
# size = 6000

## for renaming the file (optional)

In [ ]:
%%time
path = './images/'
i = 0
for filename in os.listdir(path):
    os.rename(os.path.join(path,filename), os.path.join(path,'image_'+str(i)+'.jpg'))
    i = i +1

## Video Input

### Splitting

In [2]:
def video_splitting(path,queue):
    vidcap = cv2.VideoCapture(path) # big_buck_bunny_720p_5mb.mp4

    success,image = vidcap.read()
    count = 0

    #clear directory of frames
#     dir = './frames'
#     for f in os.listdir(dir):
#         os.remove(os.path.join(dir, f))

    # success = False
    while success:
        queue.put([image,count])
#         cv2.imwrite("./frames/frame%d.jpg" % count, image)     # save frame as JPEG file    
        success,image = vidcap.read()
#         print('Read a new frame: ', success)
        count += 1
        
    print("Splitting Done and Frames obtained: ",count)

### Joining

In [3]:
def video_stitch(video_name,fps):
    
    images = ["./image_output/{}_output.jpg".format(i) for i in range(len(os.listdir('./image_output')))]
#     images = ["./frames/frame{}.jpg".format(i) for i in range(len(os.listdir('./frames')))]

    # print(len(images))
    # print(type(images[0]))
    # print(images[0])

    frame = cv2.imread(images[0])

    height, width = frame.shape[:2]

    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    video = cv2.VideoWriter(video_name, fourcc, fps, (width, height))

    for image in images:
        video.write(cv2.imread(image))
    
    cv2.destroyAllWindows()
    video.release()

## class define and model load

In [4]:
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

net = cv2.dnn.readNetFromCaffe("../MobileNetSSD_deploy.prototxt.txt", "../MobileNetSSD_deploy.caffemodel")

## function for detection

In [5]:
def prep(image, count, q):
    #image load
#     image = cv2.imread(img)
    (h, w) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)
#     index = re.sub("\D", "", img)
    q.put([blob,image,count])
    return None

def detect(queueIn):

    arr, image, num = queueIn.get()
    (h, w) = image.shape[:2]
    #load image in model
    net.setInput(arr)
    detections = net.forward()

    #look for match in image
    for i in np.arange(0, detections.shape[2]):

        #extract confidence from detected object
        confidence = detections[0, 0, i, 2]

        #filter out low confidence objects
        if confidence > min_confidence:

            idx = int(detections[0, 0, i, 1])
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])

            (startX, startY, endX, endY) = box.astype("int")

            # display the prediction
            label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
#             print("[INFO] {}".format(label))
            cv2.rectangle(image, (startX, startY), (endX, endY), COLORS[idx], 2)

            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

    # show the output image
    cv2.imwrite("./image_output/{}_output.jpg".format(num), image)
    return None

# Parallel

In [6]:
# qw = ["./frames/frame{}.jpg".format(i) for i in range(739)]
m = mp.Manager()
q_size = 100
detect_q = m.Queue(q_size)
split_q = m.Queue(q_size)

## Parallel with 3 processors

In [22]:
def fun(queueIn,queueOut,num):
    i = 0
    while True:
#         print(num,": ",i,"prep")
        
        if(i>10 and queueIn.empty()):
            print("empty prep")
            break
            
        image,count = queueIn.get()
        prep(image, count, queueOut)
        
        if(i%250 == 0):
            print(num, i,"prep done!")
                        
        i+=1
    pid = os.getpid()
    print(f'prep {pid} done')

In [23]:
def fun1(queueIn,num):
    i = 0
    while True:
#         print(num,": ",i,"detect")
        
        if(i>10 and queueIn.empty()):
            print("empty detect")
            break
            
        detect(queueIn)
        if(i%250 == 0):
            print(num,i,"detect done!")
        i+=1
    pid = os.getpid()
    print(f'detect {pid} done')

In [51]:
%%time

split_p = mp.Process(target=video_splitting, args=("big_buck_bunny_720p_5mb.mp4",split_q)) #splitting
split_p.daemon = True
split_p.start()

detect_p = mp.Process(target=fun1, args=(detect_q,1)) #detection
detect_p.daemon = True
detect_p.start()

fun(split_q,detect_q,1) #preprocessing

split_p.join()
detect_p.join()

1 0 detect done!
1 0 prep done!
1 250 prep done!
1 250 detect done!
1 500 prep done!
1 500 detect done!
Splitting Done and Frames obtained:  739
empty prep
prep 5130 done
empty detect
detect 16053 done
CPU times: user 6.78 s, sys: 4.69 s, total: 11.5 s
Wall time: 44.5 s


In [ ]:
%%time

split_p = mp.Process(target=video_splitting, args=("big_buck_bunny_720p_5mb.mp4",split_q)) #splitting
split_p.daemon = True
split_p.start()

detect_p = mp.Process(target=fun1, args=(detect_q,1)) #detection
detect_p.daemon = True
detect_p.start()

prep_p = mp.Process(target=fun, args=(split_q,detect_q,1)) #preprocessing
prep_p.daemon = True
prep_p.start()



split_p.join()
detect_p.join()
prep_p.join()

In [82]:
split_q.qsize()

0

In [83]:
detect_q.qsize()

0

## Parallel with 6 processors

In [24]:
def fps_calc(path):
    vidcap = cv2.VideoCapture(path) # big_buck_bunny_720p_5mb.mp4

    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    return fps,frame_count

In [15]:
fps_calc("big_buck_bunny_720p_5mb.mp4")

(25.0, 739)

In [25]:
def smap(f):
    return f()

def parallel(InPath,OutPath):
    
    fps,frame_count = fps_calc(InPath)
    
    split1 = functools.partial(video_splitting, InPath, split_q)
    prep1 = functools.partial(fun, split_q, detect_q,1)
    prep2 = functools.partial(fun, split_q, detect_q,2)
    dete1 = functools.partial(fun1, detect_q,1)
    dete2 = functools.partial(fun1, detect_q,2)
    dete3 = functools.partial(fun1, detect_q,3)
    pool = mp.Pool(processes=6)
    
    res = pool.map_async(smap, [split1, prep1, prep2, dete1, dete2, dete3])
    pool.close()
    pool.join()
    
    video_stitch(OutPath,fps)
    
    return None

In [26]:
%%time
parallel("big_buck_bunny_720p_5mb.mp4",'test_output/vid6.mp4')

1 0 prep done!
2 0 prep done!
1 03  0detect done!
2  detect done!
0 detect done!
1 250 prep done!
2 250 prep done!
Splitting Done and Frames obtained:  739
empty prep
prep 14168 done
empty prep
prep 14171 done
empty detect
empty detect
detect 14174 donedetect 14176 done

empty detect
detect 14179 done
CPU times: user 15.6 s, sys: 186 ms, total: 15.8 s
Wall time: 55 s
